In [16]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests

# 対象地域情報の読み込み

In [17]:
gdf_jcode = gpd.read_file('data/setup_2000_2020/jcode.geojson')
df_jcode_2000 = pd.read_csv('data/2000/jcode_2000.csv', dtype={'jcode_from': str, 'jcode_to': str})
list_jcode = df_jcode_2000['jcode_from'].to_list()
list_jcode_to = list(set(df_jcode_2000['jcode_from'].to_list()))

In [18]:
df_jcode_2000

,jcode_from,jcode_to,都道府県,政令市･郡･支庁･振興局等,市区町村
0,08220,08220,茨城県,NaN,つくば市
1,08482,08235,茨城県,筑波郡,伊奈町
2,08561,08224,茨城県,北相馬郡,守谷町
3,11203,11203,埼玉県,NaN,川口市
4,11204,11204,埼玉県,NaN,浦和市
...,...,...,...,...,...
108,14207,14207,神奈川県,NaN,茅ヶ崎市
109,14209,14209,神奈川県,NaN,相模原市
110,14213,14213,神奈川県,NaN,大和市
111,14215,14215,神奈川県,NaN,海老名市


# 地価情報の取得

In [19]:
# pd.read_excel('https://www.mlit.go.jp/totikensangyo/content/001733704.xls')
df_chika = pd.read_excel('sources/001733704.xls', sheet_name='価格推移表', header=0, skiprows=[1])
nayose = pd.read_csv('sources/chika_nayose.csv')

In [20]:
df_chika['Unnamed: 0'] = df_chika['Unnamed: 0'].astype(str)
df_chika['level'] = df_chika['Unnamed: 0'].str[0:1]
df_chika['jcode'] = df_chika['Unnamed: 0'].str[1:6]
df_chika['type'] = df_chika['Unnamed: 0'].str[-2:]
df_chika['name'] = df_chika['Unnamed: 1'].ffill()
df_chika['分類'] = df_chika['Unnamed: 2']
df_chika.loc[df_chika['jcode'] == "14150", "jcode"] = "14209" # 相模原市のjcodeを修正
df_chika = df_chika.drop(columns=['Unnamed: 0', 'Unnamed: 1','Unnamed: 2'])
df_chika = df_chika[[
    'level',   'jcode',   'type',    'name',    '分類',
    '昭和50年', '昭和51年', '昭和52年', '昭和53年', '昭和54年', '昭和55年', '昭和56年', '昭和57年',
    '昭和58年', '昭和59年', '昭和60年', '昭和61年', '昭和62年', '昭和63年', '平成元年', '平成2年',
    '平成3年',  '平成4年',  '平成5年',  '平成6年',  '平成7年',  '平成8年',  '平成9年',  '平成10年',
    '平成11年', '平成12年', '平成13年', '平成14年', '平成15年', '平成16年', '平成17年', '平成18年',
    '平成19年', '平成20年', '平成21年', '平成22年', '平成23年', '平成24年', '平成25年', '平成26年',
    '平成27年', '平成28年', '平成29年', '平成30年', '平成31年', '令和2年',  '令和3年',  '令和4年',
    '令和5年',  '令和6年'
]]

In [21]:
df_chika_res = df_chika[['jcode','分類','平成12年']].query('分類 == "住宅"').rename(columns={'平成12年':'res_price'})
df_chika_com = df_chika[['jcode','分類','平成12年']].query('分類 == "商業"').rename(columns={'平成12年':'com_price'})

df_chika_res = df_chika_res.query('jcode in @list_jcode')
df_chika_com = df_chika_com.query('jcode in @list_jcode')

In [22]:
# 2020年居住地平均価格を参考に、300,000円/坪で正規化
print('居住地価格平均値:', np.mean(df_chika_res['res_price'].to_numpy()))
print('居住地価格中央値:', np.median(df_chika_res['res_price'].to_numpy()))

居住地価格平均値: 302188.49557522126
居住地価格中央値: 250500.0


In [23]:
df_chika_res['q_i'] = df_chika_res['res_price']/300000
df_chika_com['Q_j'] = df_chika_com['com_price']/300000
print(df_chika_res['q_i'].mean())
print(df_chika_com['Q_j'].mean())

1.0072949852507374
2.221587878787879


In [24]:
df_q_i = df_chika_res[['jcode','q_i']].round(6).reset_index(drop=True)
df_Q_j = df_chika_com[['jcode','Q_j']].round(6).reset_index(drop=True)
df_Q_j.loc[df_Q_j['Q_j'].isnull(),'Q_j'] = df_q_i.loc[df_Q_j['Q_j'].isnull(),'q_i'] # Q_jがない場合はq_iで置き換える

In [25]:
df_Q_j

,jcode,Q_j
0,08220,0.596333
1,08482,0.211667
2,08561,0.438333
3,11203,1.776333
4,11204,1.926333
...,...,...
108,14205,1.933000
109,14207,1.597667
110,14213,1.401333
111,14215,1.511667


# 所得w_jの取得

In [26]:
# 所得w_jの取得
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
    'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
    'lang':'J',
    'statsDataId':'0000081782',
    # 'metaGetFlg':'N',
    'explanationGetFlg':'N',
    'annotationGetFlg':'N',
    'cdCat02':'001',
    # 'lvArea':'2-',
    'cdAreaFrom': '08000',
    'cdAreaTo': '14999',
}

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

項目名： 世帯の年間収入階級（８）、世帯の種類（２），住宅の所有の関係（５）、普通世帯数、市（６７１）・１３大都市の区（１５１）・一定規模以上の町村（２０８）
ヒット件数： 2583


[{'@id': 'cat01',
  '@name': '世帯の年間収入081231',
  'CLASS': [{'@code': '000', '@name': '総数（「不詳」を含む）', '@level': '1'},
   {'@code': '001', '@name': '\u3000２００万円未満', '@level': '1'},
   {'@code': '002', '@name': '\u3000２００\u3000～\u3000３００万円未満', '@level': '1'},
   {'@code': '003', '@name': '\u3000３００\u3000～\u3000４００', '@level': '1'},
   {'@code': '004', '@name': '\u3000４００\u3000～\u3000５００', '@level': '1'},
   {'@code': '005', '@name': '\u3000５００\u3000～\u3000７００', '@level': '1'},
   {'@code': '006', '@name': '\u3000７００\u3000～１，０００', '@level': '1'},
   {'@code': '007', '@name': '\u3000１，０００～１，５００', '@level': '1'},
   {'@code': '008', '@name': '\u3000１，５００万円以上', '@level': '1'}]},
 {'@id': 'cat02',
  '@name': '世帯の種類２081233',
  'CLASS': {'@code': '001', '@name': '総数', '@level': '1', '@unit': '世帯'}},
 {'@id': 'area',
  '@name': '市区一定規模080051',
  'CLASS': [{'@code': '08201', '@name': '水戸市', '@level': '1'},
   {'@code': '08202', '@name': '日立市', '@level': '1'},
   {'@code': '08203', '@name': '土浦市', '@

In [27]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])

def parse_wage(x):
   if   x=='001': return 100
   elif x=='002': return 250
   elif x=='003': return 350
   elif x=='004': return 450
   elif x=='005': return 600
   elif x=='006': return 850
   elif x=='007': return 1250
   elif x=='008': return 2000
   else: return 0

def parse_num(x):
   if x=='-': return 0
   else: return int(x)

df['weight'] = df['@cat01'].apply(parse_wage)
df['num'] = df['$'].astype(int)
df['jcode'] = df['@area']


wm = lambda x: np.average(x, weights=df.loc[x.index, "num"]).round(2)
df_w_j = df.groupby(by='jcode').agg(w_j=("weight", wm)).loc[list_jcode_to].reset_index().sort_values('jcode')

In [28]:
# 2020年賃金率を参考に、300万円/年で正規化
print('賃金率平均値:', np.mean(df_w_j['w_j'].to_numpy()).round(2))
print('賃金率中央値:', np.median(df_w_j['w_j'].to_numpy()).round(2))

賃金率平均値: 310.4
賃金率中央値: 304.43


In [29]:
df_w_j['w_j'] = (df_w_j['w_j']/300).round(6)
df_w_j

,jcode,w_j
85,08220,0.970733
19,08482,1.154167
99,08561,1.239167
24,11203,0.957567
91,11204,1.078267
...,...,...
112,14207,1.112867
29,14209,1.003167
76,14213,1.018700
68,14215,1.122900


# 通勤割合λ_ijの取得

In [30]:
From = '08000'
To = '14999'

URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000033333',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     # 'cdArea':','.join(list),
     # # 'cdCat02':','.join(list)
     # 'lvArea':'4-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
     # 'lvCat02':'4-',
     'cdCat01':'000',
     'cdCat02':'T01',
     'cdCat03':'001',
     }

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])

df_1 = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_1 = df_1[['@area', '$']].rename(columns={'@area':'from', '$':'value'})
df_1['to'] = df_1['from']
df_1 = df_1[['to','from', 'value']]
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

項目名： 男女（３）、年齢（５歳階級２０Ｃ）、常住地による人口（４Ｂ）、１５歳以上就業者数 （常住地）全国・都道府県・市区町村
ヒット件数： 514


[{'@id': 'cat01',
  '@name': '男女031123',
  'CLASS': {'@code': '000', '@name': '総数', '@level': '1'}},
 {'@id': 'cat02',
  '@name': '年齢階級031187',
  'CLASS': {'@code': 'T01', '@name': '総数', '@level': '1'}},
 {'@id': 'cat03',
  '@name': '常住地人口031318',
  'CLASS': {'@code': '001',
   '@name': '\u3000うち自宅外の自市区町村で従業',
   '@level': '1',
   '@unit': '人'}},
 {'@id': 'area',
  '@name': '常住地030213',
  'CLASS': [{'@code': '08000', '@name': '茨城県', '@level': '2'},
   {'@code': '08201', '@name': '水戸市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08202', '@name': '日立市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08203', '@name': '土浦市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08204', '@name': '古河市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08205', '@name': '石岡市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08206', '@name': '下館市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08207', '@name': '結城市', '@level': '4', '@parentCode': '08000'},
 

In [31]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000033343',
     'startPosition':'1',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     # 'cdArea':','.join(list),
     # # 'cdCat02':','.join(list)
     # 'lvArea':'4-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
     # 'lvCat02':'4-',
     'cdCat01From':From,
     'cdCat01To':To,
     'cdCat02':'000',
     'cdCat03':'001'
     }

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])

df_2 = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_2 = df_2[['@cat01', '@area', '$']].rename(columns={'@cat01':'to', '@area':'from', '$':'value'})
df_2 = df_2[df_2['from'] != df_2['to']]
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

項目名： 男女（３）、通勤・通学者数（２）、１５歳以上就業者数・１５歳以上通学者数 （常住地）市区町村、（従通地）都道府県・市区町村
ヒット件数： 85770


[{'@id': 'cat01',
  '@name': '従業通学地030220',
  'CLASS': [{'@code': '08000', '@name': '茨城県', '@level': '2'},
   {'@code': '08201', '@name': '水戸市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08202', '@name': '日立市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08203', '@name': '土浦市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08204', '@name': '古河市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08205', '@name': '石岡市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08206', '@name': '下館市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08207', '@name': '結城市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08208', '@name': '龍ケ崎市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08210', '@name': '下妻市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08211', '@name': '水海道市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08212', '@name': '常陸太田市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08214', '@name': 

In [32]:
df = pd.concat([df_1, df_2])

df_commute_ij = pd.DataFrame(0,columns=list_jcode,index=list_jcode)
for i in df.itertuples():
    if (i[1] not in list_jcode)|(i[2] not in list_jcode): continue
    if i[3] in ['-',None]: 
        df_commute_ij.at[i[2],i[1]]=0
    else: 
        if int(i[3]) > 100: df_commute_ij.at[i[2],i[1]]=int(i[3])
        # df_commute_ij.at[i[2],i[1]]=int(i[3])

In [33]:
df_commute_ij

,08220,08482,08561,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
08220,47571,357,307,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
08482,986,3201,777,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
08561,811,305,7582,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11203,0,0,0,92029,8812,3604,369,505,738,2505,...,0,0,0,0,0,0,0,0,0,0
11204,0,0,0,10264,74930,11156,833,1356,4124,771,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2388,12275,33984,523,626,887,234
14209,0,0,0,0,0,0,0,0,0,0,...,1047,744,1449,305,1968,325,141799,5026,2530,5583
14213,0,0,0,0,0,0,0,0,0,0,...,228,382,382,472,5155,360,3658,36591,2007,2604
14215,0,0,0,0,0,0,0,0,0,0,...,137,101,176,150,1569,428,1737,2249,17404,1721


# 世帯あたり子供の数n_ijの取得

In [34]:
URL = 'http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData'
p = {
     'appId':'6f812f8eb28dc679954f0eef1472fca422edc08d',
     'lang':'J',
     'statsDataId':'0000032965',
     # 'metaGetFlg':'N',
     'explanationGetFlg':'N',
     'annotationGetFlg':'N',
     'cdCat01':'00700',
     'cdCat02':'000',
     'cdCat03':','.join(['200','201','202']),
     'cdCat04':'000',
     # 'lvArea':'5-',
     'cdAreaFrom': From,
     'cdAreaTo': To,
}

res = requests.get(URL,p)
text = res.json()
print('項目名：',text['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']['TITLE']['$'])
print('ヒット件数：',text['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['TOTAL_NUMBER'])
text['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

項目名： 国籍（２）、年齢各歳階級（１２３）、男女（３）、人口 市区町村、全域・人口集中地区の別
ヒット件数： 1773


[{'@id': 'cat01',
  '@name': '全域・集中の別030184',
  'CLASS': {'@code': '00700', '@name': '全域', '@level': '1'}},
 {'@id': 'cat02',
  '@name': '国籍031121',
  'CLASS': {'@code': '000', '@name': '総数（外国，不詳を含む）', '@level': '1'}},
 {'@id': 'cat03',
  '@name': '年齢各歳１２３030798',
  'CLASS': [{'@code': '200',
    '@name': '0～4歳',
    '@level': '2',
    '@parentCode': 'T01'},
   {'@code': '201', '@name': '5～9歳', '@level': '2', '@parentCode': 'T01'},
   {'@code': '202', '@name': '10～14歳', '@level': '2', '@parentCode': 'T01'}]},
 {'@id': 'cat04',
  '@name': '男女031123',
  'CLASS': {'@code': '000', '@name': '総数', '@level': '1', '@unit': '人'}},
 {'@id': 'area',
  '@name': '県市区町村030172',
  'CLASS': [{'@code': '08000', '@name': '茨城県', '@level': '2'},
   {'@code': '08001', '@name': '茨城県市部', '@level': '3', '@parentCode': '08000'},
   {'@code': '08002', '@name': '茨城県郡部', '@level': '3', '@parentCode': '08000'},
   {'@code': '08201', '@name': '水戸市', '@level': '4', '@parentCode': '08000'},
   {'@code': '08202', '@na

In [35]:
df = pd.DataFrame(text['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
df_children_i = pd.DataFrame(0,index=list_jcode,columns=[0])
for i in df.itertuples():
    try: 
        if i[8] == '-': pass
        else: df_children_i.at[i[5],0] += int(i[8])
    except: pass

arr_children_ij = df_children_i.to_numpy() * df_commute_ij.to_numpy()/df_commute_ij.sum(axis=1).to_numpy().reshape(1,-1).T
df_children_ij = pd.DataFrame(arr_children_ij,index=list_jcode, columns=list_jcode)
df_children_ij = (df_children_ij).fillna(0)

In [36]:
df_children_ij

,08220,08482,08561,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
08220,26257.129922,197.048525,169.450692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
08482,509.864997,1655.251374,401.790165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
08561,492.536777,185.232696,4604.702638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11203,0.000000,0.000000,0.000000,30429.657350,2913.713509,1191.673115,122.010927,166.979723,244.021853,828.285558,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11204,0.000000,0.000000,0.000000,3660.298187,26721.175285,3978.398925,297.060443,483.570181,1470.680994,274.950302,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,902.119202,4637.149584,12838.198896,197.574683,236.485184,335.083640,88.398615
14209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,370.130803,263.015585,512.244062,107.822249,695.718644,114.892561,50128.154455,1776.769260,894.394395,1973.677433
14213,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,76.982352,128.979204,128.979204,159.366974,1740.543965,121.551082,1235.094049,12354.654554,677.647282,879.219493
14215,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,57.231479,42.192550,73.523652,62.662203,655.446646,178.796153,725.628314,939.515301,7270.486573,718.944346


# 交通費用の算出

In [37]:
gdf_gov = gpd.read_file('data/setup_2000_2020/gov_poi.geojson',dtypes={'jcode':str})

In [38]:
df_time = pd.read_csv('sources/e-2_H10.csv', header=3, encoding='CP932')
df_time.columns = ['origin','destination','発ゾーン','着ゾーン','鉄道','路線バス・都電','自動車','２輪車','自転車','徒歩','その他','合計']
df_time['発ゾーン'] = df_time['発ゾーン'].str.strip(':').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
df_time['着ゾーン'] = df_time['着ゾーン'].str.strip(':').str.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
ptcode = gdf_gov['H10'].tolist()
df_time = df_time[(df_time['発ゾーン'].isin(ptcode))&(df_time['着ゾーン'].isin(ptcode))]

In [39]:
df_time = df_time.merge(gdf_gov[['jcode','H10']], left_on='発ゾーン', right_on='H10', how='left').rename(columns={'jcode':'ori_jcode'}).drop(columns='H10')
df_time = df_time.merge(gdf_gov[['jcode','H10']], left_on='着ゾーン', right_on='H10', how='left').rename(columns={'jcode':'dst_jcode'}).drop(columns='H10')

df_time = df_time.sort_values(['ori_jcode', 'dst_jcode'])
df_time['鉄道'] = df_time['鉄道'].where(df_time['鉄道'] !=0.0, df_time[['路線バス・都電','自動車']].apply(lambda x: np.round(np.mean(x),2), axis=1))

In [40]:
df_time_ij = (df_time.pivot_table(values='合計', index='ori_jcode', columns='dst_jcode', fill_value=540)/(60*24)).round(6)

In [41]:
df_time_ij

dst_jcode,08220,08224,08235,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
ori_jcode,,,,,,,,,,,,,,,,,,,,,
08220,0.009201,0.025729,0.013646,0.375000,0.375000,0.375000,0.375000,0.118056,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
08224,0.025875,0.007590,0.014903,0.069444,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
08235,0.013563,0.015312,0.009653,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
11203,0.375000,0.072917,0.375000,0.007306,0.028188,0.030382,0.062500,0.036979,0.038688,0.056590,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
11204,0.375000,0.375000,0.375000,0.028722,0.008090,0.019500,0.053729,0.036299,0.010854,0.014799,...,0.045139,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.083333,0.086806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0.375000,0.375000,0.375000,0.375000,0.375000,0.086806,0.027778,0.375000,0.375000,0.375000,...,0.055556,0.073229,0.059722,0.038896,0.022986,0.009812,0.063271,0.054562,0.033194,0.051479
14209,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.065326,0.375000,0.375000,0.375000,...,0.037812,0.056056,0.037146,0.041667,0.052257,0.060340,0.009819,0.034326,0.036708,0.028819
14213,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.038097,0.036208,0.031806,0.053465,0.033729,0.045069,0.034736,0.008535,0.025972,0.014271


In [42]:
df_time_ij_sim = df_time_ij
df_time_ij_sim.loc['13101':'13123',:] = df_time_ij_sim.loc['13101':'13123',:]*0.50
df_time_ij_sim.loc[:,'13101':'13123'] = df_time_ij_sim.loc[:,'13101':'13123']*0.50
df_time_ij_sim.loc['13101':'13123','13101':'13123'] = df_time_ij_sim.loc['13101':'13123','13101':'13123']/0.50
df_time_ij_sim

dst_jcode,08220,08224,08235,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
ori_jcode,,,,,,,,,,,,,,,,,,,,,
08220,0.009201,0.025729,0.013646,0.375000,0.375000,0.375000,0.375000,0.118056,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
08224,0.025875,0.007590,0.014903,0.069444,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
08235,0.013563,0.015312,0.009653,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
11203,0.375000,0.072917,0.375000,0.007306,0.028188,0.030382,0.062500,0.036979,0.038688,0.056590,...,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
11204,0.375000,0.375000,0.375000,0.028722,0.008090,0.019500,0.053729,0.036299,0.010854,0.014799,...,0.045139,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.083333,0.086806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0.375000,0.375000,0.375000,0.375000,0.375000,0.086806,0.027778,0.375000,0.375000,0.375000,...,0.055556,0.073229,0.059722,0.038896,0.022986,0.009812,0.063271,0.054562,0.033194,0.051479
14209,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.065326,0.375000,0.375000,0.375000,...,0.037812,0.056056,0.037146,0.041667,0.052257,0.060340,0.009819,0.034326,0.036708,0.028819
14213,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,...,0.038097,0.036208,0.031806,0.053465,0.033729,0.045069,0.034736,0.008535,0.025972,0.014271


運賃の参考
初乗り:　130円 (第84条)
距離運賃: 16.8円/km (第77条)
https://www.desktoptetsu.com/ryoki/ryokikaitei_jre_2.htm

In [43]:
gdf_gov = gdf_gov.to_crs(epsg=6677)
list_gov = gdf_gov['jcode'].to_list()
distance_matrix = gdf_gov.geometry.apply(lambda x: gdf_gov.distance(x)).values
distance_matrix = np.where(distance_matrix/1000 > 130/16.8, distance_matrix/1000*16.8, 130)
distance_matrix = distance_matrix * (22*12) / 3000000
distance_matrix = np.round(distance_matrix, 5)
df_cost_ij = pd.DataFrame(distance_matrix,index=list_gov, columns=list_gov)

In [44]:
df_cost_ij

,08220,08224,08235,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
08220,0.01144,0.02553,0.02047,0.06523,0.06812,0.06640,0.09350,0.06670,0.06841,0.05569,...,0.10251,0.10486,0.10956,0.14401,0.14508,0.15230,0.12602,0.12803,0.13900,0.13233
08224,0.02553,0.01144,0.01144,0.04099,0.04650,0.04685,0.07209,0.05115,0.04793,0.03070,...,0.07754,0.07964,0.08481,0.11851,0.11955,0.12687,0.10182,0.10285,0.11391,0.10737
08235,0.02047,0.01144,0.01144,0.04891,0.05491,0.05527,0.08047,0.05923,0.05636,0.03828,...,0.08485,0.08667,0.09227,0.12432,0.12576,0.13346,0.10954,0.10990,0.12104,0.11462
11203,0.06523,0.04099,0.04891,0.01144,0.01371,0.02059,0.03417,0.03286,0.01811,0.01144,...,0.03771,0.04079,0.04442,0.08358,0.08299,0.08873,0.06090,0.06347,0.07419,0.06734
11204,0.06812,0.04650,0.05491,0.01371,0.01144,0.01144,0.02563,0.02030,0.01144,0.02218,...,0.04113,0.04546,0.04613,0.08984,0.08808,0.09227,0.05983,0.06619,0.07605,0.06885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0.15230,0.12687,0.13346,0.08873,0.09227,0.09856,0.07688,0.10852,0.09495,0.09685,...,0.05140,0.04794,0.04629,0.01924,0.01164,0.01144,0.03919,0.02619,0.01856,0.02539
14209,0.12602,0.10182,0.10954,0.06090,0.05983,0.06468,0.03956,0.07280,0.06140,0.07126,...,0.02652,0.02768,0.01853,0.04747,0.04129,0.03919,0.01144,0.01785,0.02064,0.01435
14213,0.12803,0.10285,0.10990,0.06347,0.06619,0.07239,0.05122,0.08236,0.06879,0.07234,...,0.02577,0.02323,0.02011,0.03006,0.02477,0.02619,0.01785,0.01144,0.01144,0.01144
14215,0.13900,0.11391,0.12104,0.07419,0.07605,0.08186,0.05886,0.09120,0.07835,0.08333,...,0.03653,0.03438,0.03005,0.02956,0.02219,0.01856,0.02064,0.01144,0.01144,0.01144


# その他の指標

In [45]:
df_p_i = pd.DataFrame({
    'jcode': list_jcode,
    'p_i':1
})

In [46]:
df_param = pd.DataFrame({
    'alpha': [0.8],
    'gamma': [0.375],
    'psi': [0.25],
    'beta_cns': [0.65],
    'beta_flr': [0.175], 
    'beta_chd': [0.175]
})

In [47]:
df_scaler = pd.DataFrame({
    'T': [1],
    'L': [0.66],
    'N': [np.sum(df_commute_ij.to_numpy())],
    'mu_cost': [0.04],
    'mu_room': [0.02]
})

# 対象年度のマトリックスから、対象地域コードに置き換えてAggregateする

In [48]:
def agg_sum_df(df):
    df_ij = df.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to')
    df_ij = df_ij.T.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to').T
    df_ij = df_ij.groupby(df_ij.index).sum().T.groupby(df_ij.index).sum().T
    return df_ij

def agg_mean_df(df):
    df_ij = df.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to')
    df_ij = df_ij.T.merge(df_jcode_2000[['jcode_from', 'jcode_to']], left_index=True, right_on='jcode_from').drop(columns=['jcode_from']).set_index('jcode_to').T
    df_ij = df_ij.groupby(df_ij.index).mean().T.groupby(df_ij.index).mean().T
    return df_ij

In [49]:
df_commute_ij = agg_sum_df(df_commute_ij)
df_children_ij = agg_sum_df(df_children_ij)
df_n_ij = (df_children_ij/df_commute_ij).fillna(0).round(6)
df_Pi_ij = (df_commute_ij/np.sum(df_commute_ij.to_numpy())).round(9)

In [50]:
df_Pi_ij

,08220,08224,08235,11203,11204,11205,11208,11219,11220,11221,...,14135,14136,14137,14204,14205,14207,14209,14213,14215,14216
08220,0.004536,0.000029,0.000034,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
08224,0.000077,0.000723,0.000029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
08235,0.000094,0.000074,0.000305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11203,0.000000,0.000000,0.000000,0.008775,0.000840,0.000344,0.000035,0.000048,0.000070,0.000239,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11204,0.000000,0.000000,0.000000,0.000979,0.007145,0.001064,0.000079,0.000129,0.000393,0.000074,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14207,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000228,0.001170,0.003240,0.000050,0.000060,0.000085,0.000022
14209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000100,0.000071,0.000138,0.000029,0.000188,0.000031,0.013521,0.000479,0.000241,0.000532
14213,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000022,0.000036,0.000036,0.000045,0.000492,0.000034,0.000349,0.003489,0.000191,0.000248
14215,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000013,0.000010,0.000017,0.000014,0.000150,0.000041,0.000166,0.000214,0.001660,0.000164


# データの保存

In [51]:
df_merged = df_jcode_2000.merge(df_w_j, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_q_i, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_Q_j, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.merge(df_p_i, left_on='jcode_from', right_on='jcode', how='left').drop(columns=['jcode'])
df_merged = df_merged.groupby(by='jcode_to').agg({'w_j':'mean','q_i':'mean','Q_j':'mean','p_i':'mean'}).reset_index(names='jcode')

In [56]:
df_scaler.to_csv('data/2000/scaler.csv', index=False)
df_Pi_ij.to_csv('data/2000/Pi_ij.csv')
df_n_ij.to_csv('data/2000/n_ij.csv')

In [57]:
df_scaler.to_csv('data/2000/scaler.csv', index=False)
df_param.to_csv('data/2000/param.csv', index=False)

df_merged[['jcode']].to_csv('data/2000/jcode.csv')
df_merged[['jcode','w_j']].to_csv('data/2000/w_j.csv', index=False)
df_merged[['jcode','p_i']].to_csv('data/2000/p_i.csv', index=False)
df_merged[['jcode','q_i']].to_csv('data/2000/q_i.csv', index=False)
df_merged[['jcode','Q_j']].to_csv('data/2000/Q_j.csv', index=False)

df_Pi_ij.to_csv('data/2000/Pi_ij.csv')
df_n_ij.to_csv('data/2000/n_ij.csv')
df_time_ij.to_csv('data/2000/t_ij.csv')
df_cost_ij.to_csv('data/2000/tau_ij.csv')

# 可視化

In [ ]:
df_merged

In [52]:
# 通勤人数の集計
df_commute_ij_agg = pd.DataFrame({
    'jcode': df_commute_ij.index,
    'residents': df_commute_ij.sum(axis=1, numeric_only=True).to_numpy(),
    'workers': df_commute_ij.sum(axis=0, numeric_only=True).to_numpy()
})

# 子供の数の集計
df_children_ij_agg = pd.DataFrame({
    'jcode': df_children_ij.index,
    'children_i': df_children_ij.sum(axis=1, numeric_only=True).to_numpy(),
    'children_j': df_children_ij.sum(axis=0, numeric_only=True).to_numpy(),
})

# 子供の数の平均
df_n_ij_agg = pd.DataFrame({
    'jcode': df_n_ij.index,
    'n_i_ave': df_n_ij[df_n_ij != 0].mean(axis=1, numeric_only=True).to_numpy(),
    'n_j_ave': df_n_ij[df_n_ij != 0].mean(axis=0, numeric_only=True).to_numpy(),
})

df_merged = df_merged.merge(df_commute_ij_agg, on='jcode', how='left')
df_merged = df_merged.merge(df_children_ij_agg, on='jcode', how='left')
df_merged = df_merged.merge(df_n_ij_agg, on='jcode', how='left')

In [53]:
df_merged.describe().round(3).transpose()[['mean', 'min', '25%', '50%', '75%', 'max']]

,mean,min,25%,50%,75%,max
w_j,1.035,0.802,0.966,1.015,1.095,1.431
q_i,1.010,0.212,0.725,0.835,1.096,4.519
Q_j,2.192,0.212,1.300,1.768,2.256,12.074
p_i,1.000,1.000,1.000,1.000,1.000,1.000
residents,94480.153,6548.000,49404.500,72428.000,114271.500,357272.000
workers,94480.153,3863.000,31067.000,51349.000,115533.000,647627.000
children_i,28625.018,3386.000,15449.000,21854.000,32168.000,89531.000
children_j,28625.018,2037.533,10660.239,16759.682,34355.613,187911.066
n_i_ave,0.320,0.181,0.278,0.322,0.352,0.607
n_j_ave,0.320,0.283,0.306,0.314,0.324,0.559


In [72]:
gdf = gpd.read_file('data/setup_2000_2020/jcode.geojson')
gdf['jcode'] = gdf['jcode'].astype(str)
gdf = gdf.merge(df_merged, left_on='jcode', right_on='jcode', how='left')

In [90]:
from libs_QSM.Plot_gdf import plot_gdf

In [89]:
import os
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
gdf_KEN = gpd.read_file(os.path.join(os.getcwd(), 'sources','japan_ver84/japan_ver84.shp'))
gdf_KEN = gdf_KEN.dissolve('KEN').query('KEN in ["茨城県","埼玉県", "千葉県", "東京都", "神奈川県"]')

In [94]:
def plot_gdf(given_gdf, dir, col, unit, format):
    top = max(abs(given_gdf[col]))*1.1
    if min(given_gdf[col]) < 0 :
        norm = Normalize(vmin=-top, vmax=top)
        cmap = 'seismic_r'
    else: 
        norm = Normalize(vmin=0, vmax=top)
        cmap = 'Blues'
    fig = given_gdf.plot(
        column=col,
        norm = norm,
        legend=True,
        edgecolor='black',  # 境界線の色
        linewidth=0.5,   
        figsize=(8, 8),
        cmap=cmap,
        legend_kwds={'shrink': 0.65, 'format':format},
        missing_kwds={'color': 'white', "hatch": "///", "edgecolor": "black"}
    )
    fig = gdf_KEN.plot(ax=fig, color=(0,0,0,0), edgecolor='black')
    clb = fig.get_figure().axes[-1]
    clb.set_title(col)
    plt.text(140.35, 35.2, unit, horizontalalignment='right',verticalalignment='bottom')
    fig.set_xlim(139.2, 140.2)
    fig.set_ylim(35.25, 36.25)
    plt.savefig(f"images/{dir}/{col}.png", format='png',bbox_inches="tight",pad_inches=0.1)

In [ ]:
plot_gdf(gdf, '2000', 'w_j', '[万円]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'q_i', '[万円]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'Q_j', '[万円]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'residents', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'workers', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'children_i', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'children_j', '[万人]', lambda x, pos: int(x/10000))

In [ ]:
plot_gdf(gdf, '2000', 'n_i_ave', '[人]', lambda x, pos: x.round(2))

In [ ]:
plot_gdf(gdf, '2000', 'n_j_ave', '[人]', lambda x, pos: x.round(2))